In [1]:
import os
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("spark-test") \
    .master(os.getenv('SPARK_MASTER_URL')) \
    .config('spark.driver.cores', '1')\
    .config('spark.driver.memory', '1G')\
    .config('spark.executor.instances', '1')\
    .config('spark.executor.cores', '1')\
    .config('spark.executor.memory', '512m')\
    .config("spark.ui.port", "4045")\
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.791.jar,/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.10.1.jar") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "nessie") \
    .config("spark.sql.catalog.iceberg.uri", "http://nessie-catalog:19120/api/v2") \
    .config("spark.sql.catalog.iceberg.ref", "main") \
    .config("spark.sql.catalog.iceberg.warehouse", "warehouse") \
    .config("spark.sql.catalog.iceberg.authentication.type", "NONE") \
    .config("spark.sql.catalog.iceberg.s3.endpoint", "http://ozone-s3g:9878") \
    .config("spark.sql.catalog.iceberg.s3.access-key-id", "ozone-access-key") \
    .config("spark.sql.catalog.iceberg.s3.secret-access-key", "ozone-secret-key") \
    .config("spark.sql.catalog.iceberg.s3.path-style-access", "true") \
    .config("spark.sql.defaultCatalog", "iceberg") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.defaultFS", "s3a://ozone-om") \
    .config("spark.hadoop.fs.s3a.access.key", "ozone-access-key") \
    .config("spark.hadoop.fs.s3a.secret.key", "ozone-secret-key") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://ozone-s3g:9878") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.committer.name", "directory") \
    .config("spark.hadoop.fs.s3a.committer.staging.tmp.path", "/tmp/staging") \
    .config("spark.hadoop.fs.s3a.multiobjectdelete.enable", "false") \
    .config("spark.hadoop.fs.s3a.directory.marker.retention", "keep") \
    .config("spark.hadoop.fs.s3a.connection.establish.timeout", "5000") \
    .config("spark.hadoop.fs.s3a.connection.timeout", "60000") \
    .config("spark.hadoop.fs.s3a.socket.timeout", "60000") \
    .config("spark.hadoop.fs.s3a.attempts.maximum", "50") \
    .config("spark.hadoop.fs.s3a.retry.limit", "50") \
    .config("spark.hadoop.fs.s3a.retry.interval", "5s") \
    .getOrCreate()

26/02/12 17:04:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark.sql("SHOW CATALOGS;").show()

+-------------+
|      catalog|
+-------------+
|      iceberg|
|spark_catalog|
+-------------+



In [4]:
spark.sql("CREATE DATABASE IF NOT EXISTS iceberg.test_iceberg_db_2 LOCATION 's3a://mybucket/warehouse/test_iceberg_db_2'")

DataFrame[]

In [5]:
spark.sql("SHOW DATABASES IN iceberg").show()

+-----------------+
|        namespace|
+-----------------+
|test_iceberg_db_2|
|  test_iceberg_db|
+-----------------+



In [10]:
spark.sql("SHOW TABLES IN iceberg.test_iceberg_db_2").show()

+-----------------+------------------+-----------+
|        namespace|         tableName|isTemporary|
+-----------------+------------------+-----------+
|test_iceberg_db_2|test_iceberg_table|      false|
+-----------------+------------------+-----------+



In [7]:
df = spark.createDataFrame([(1, "tgete", "2025-02-01"), (3, "vdsv", "2025-06-01")], ["id", "descr", "report_dt"])

In [8]:
df.show()

+---+-----+----------+
| id|descr| report_dt|
+---+-----+----------+
|  1|tgete|2025-02-01|
|  3| vdsv|2025-06-01|
+---+-----+----------+



In [9]:
spark.sql("""
CREATE TABLE IF NOT EXISTS iceberg.test_iceberg_db_2.test_iceberg_table (
    id INT,
    descr STRING,
    report_dt STRING
) USING iceberg
PARTITIONED BY (report_dt)
""")

26/02/12 17:07:02 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
26/02/12 17:07:02 WARN VersionInfoUtils: The AWS SDK for Java 1.x entered maintenance mode starting July 31, 2024 and will reach end of support on December 31, 2025. For more information, see https://aws.amazon.com/blogs/developer/the-aws-sdk-for-java-1-x-is-in-maintenance-mode-effective-july-31-2024/
You can print where on the file system the AWS SDK for Java 1.x core runtime is located by setting the AWS_JAVA_V1_PRINT_LOCATION environment variable or aws.java.v1.printLocation system property to 'true'.
This message can be disabled by setting the AWS_JAVA_V1_DISABLE_DEPRECATION_ANNOUNCEMENT environment variable or aws.java.v1.disableDeprecationAnnouncement system property to 'true'.
The AWS SDK for Java 1.x is being used here:
at java.base/java.lang.Thread.getStackTrace(Thread.java:1619)
at com.amazonaws.util.VersionInfoUtils.printDeprecationAn

DataFrame[]

In [12]:
# insert into table
df.write\
    .mode("append") \
    .insertInto("iceberg.test_iceberg_db_2.test_iceberg_table")

In [13]:
spark \
    .table("iceberg.test_iceberg_db_2.test_iceberg_table") \
    .show()

+---+-----+----------+
| id|descr| report_dt|
+---+-----+----------+
|  1|tgete|2025-02-01|
|  1|tgete|2025-02-01|
|  3| vdsv|2025-06-01|
|  3| vdsv|2025-06-01|
+---+-----+----------+



In [14]:
# write file to S3
df.write \
    .mode("overwrite") \
    .format("parquet") \
    .save("s3a://mybucket/test_db/spark_test/test.parquet")

In [15]:
spark.read \
    .parquet("s3a://mybucket/test_db/spark_test/test.parquet") \
    .show()

+---+-----+----------+
| id|descr| report_dt|
+---+-----+----------+
|  1|tgete|2025-02-01|
|  3| vdsv|2025-06-01|
+---+-----+----------+



In [16]:
spark.stop()